<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Проект-для-«Викишоп»-c-BERT" data-toc-modified-id="Проект-для-«Викишоп»-c-BERT-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Проект для «Викишоп» c BERT</a></span><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Настройка-окружения" data-toc-modified-id="Настройка-окружения-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Настройка окружения</a></span></li><li><span><a href="#Чтение-файла" data-toc-modified-id="Чтение-файла-1.1.2"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>Чтение файла</a></span></li><li><span><a href="#Подготовка-корпусов" data-toc-modified-id="Подготовка-корпусов-1.1.3"><span class="toc-item-num">1.1.3&nbsp;&nbsp;</span>Подготовка корпусов</a></span></li><li><span><a href="#BERT" data-toc-modified-id="BERT-1.1.4"><span class="toc-item-num">1.1.4&nbsp;&nbsp;</span>BERT</a></span></li><li><span><a href="#Подготовка-выборок" data-toc-modified-id="Подготовка-выборок-1.1.5"><span class="toc-item-num">1.1.5&nbsp;&nbsp;</span>Подготовка выборок</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Результаты-исследования" data-toc-modified-id="Результаты-исследования-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Результаты исследования</a></span><ul class="toc-item"><li><span><a href="#Общие-выводы" data-toc-modified-id="Общие-выводы-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Общие выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></li></ul></li></ul></div>

# Проект для «Викишоп» c BERT
<p>
<div align="right"><b>Спринт 12 | Когорта ДС13 | Артур Урусов</b></div>

<blockquote>
<p>Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

<p>Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.
    
<p> Постройте модель со значением метрики качества *F1* не меньше 0.75. 
</blockquote>

## Подготовка

### Настройка окружения

In [1]:
import re
import nltk
import torch
import warnings
import transformers
import numpy as np
import pandas as pd
from tqdm import notebook
from catboost import CatBoostClassifier
from nltk.corpus import stopwords
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from keras.preprocessing.text import text_to_word_sequence

In [2]:
STATE = 42
warnings.simplefilter('ignore')

Проверим, настроена ли CUDA для работы с BERT:

In [3]:
# Если доступен GPU...
if torch.cuda.is_available():        
    # Скажем PyTorch использовать GPU.    
    device = torch.device("cuda")    
    print('There are %d GPU(s) available.' % torch.cuda.device_count())    
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# Если нет...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: GeForce RTX 2070 SUPER


### Чтение файла

Прочитаем данные в датафрейм.

In [4]:
df = pd.read_csv('datasets\\toxic_comments.csv')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


Проверим, нет ли лишних значений в столбце `toxic`.

In [5]:
df['toxic'].unique()

array([0, 1], dtype=int64)

Изучим баланс классов:

In [6]:
df.groupby('toxic').count()

,text
toxic,
0,143346
1,16225


Как видим, токсичные комментарии составляют всего лишь около 10% от датасета.

Проверим сколько строк с латиницей, и сколько с кириллицей:

In [7]:
print('Latin text:', df[df['text'].str.findall(r'[a-zA-Z]').str.len() > 0].shape[0])
print('Cyrillic text:', df[df['text'].str.findall(r'[а-яА-ЯёЁ]').str.len() > 0].shape[0])

Latin text: 159564
Cyrillic text: 259


Как видим, подавляющее большинство текста на латинице, то есть скорее всего датасет на английском языке.

Сделаем выборку для работы с BERT, но выровняем баланс классов:

In [8]:
ones = df[df['toxic'] == 1]
zeros = df[df['toxic'] == 0].sample(len(ones))
sample = pd.concat([ones, zeros], axis=0)
sample = sample.sample(frac=1)
sample.reset_index(drop=True, inplace=True)
sample

,text,toxic
0,"Allright, the fact that I don't agree with you...",0
1,Naughty naughty u slimy little greaseball. \n\...,1
2,It's like watching a rabid poodle try to bite ...,1
3,"UFC 191 \n\nTHIS IS THE FINALIZED BOUT ORDER, ...",1
4,"""\nGreat! Thanks ;) TALK! """,0
...,...,...
32445,God I'm fucking scared I won't be able to vand...,1
32446,FUCK THA WORLD AND FUCK SINNEED AND FUCK FT2,1
32447,Don't even start with me Mr. Confused Gender. ...,1
32448,Banning Fut.Perf. ☼ from this Page \n\nThis se...,1


Проверим баланс классов:

In [9]:
sample.groupby('toxic').count()

,text
toxic,
0,16225
1,16225


Баланс классов соблюдён.

### Подготовка корпусов

Создадим корпус:

In [10]:
corpus_X = df['text'].values.astype('U')
corpus_y = df['toxic']
corpus_X_train, corpus_X_test, corpus_y_train, corpus_y_test = train_test_split(
    corpus_X, corpus_y, random_state=STATE, test_size=.2
)
print(corpus_X_train.shape, corpus_y_train.shape, corpus_X_test.shape, corpus_y_test.shape)

(127656,) (127656,) (31915,) (31915,)


Подгрузим словарь стоп-слов английского языка и напишем функции для очищения и токенизации текста, а также для разделения нескольких наборов данных на выборки.

In [11]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def clean_text(text):
    t = str.maketrans(dict.fromkeys("'`'", ""))
    text = text.translate(t)
    return ' '.join(re.sub(r'[^a-zA-Z ]', ' ', text).split())


def keras_tokenize(text):
    text = clean_text(text)
    tokens = text_to_word_sequence(text)
    return ' '.join(tokens)


def multi_split(X, y, random_state=None, test_size=.2, stratify=None):
    X_train, X_test, y_train, y_test = [], [], [], []
    
    for i in range(len(X)):
        X_tr, X_t, y_tr, y_t = train_test_split(
            X[i], y[i], random_state=random_state, test_size=test_size, stratify=stratify[i]
        )
        
        X_train.append(X_tr)
        X_test.append(X_t)
        y_train.append(y_tr)
        y_test.append(y_t)
    
    return X_train, X_test, y_train, y_test

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\arthr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Создадим очищенный корпус.

In [12]:
clean_corpus_X_train = []
for i in notebook.tqdm(range(len(corpus_X_train))):
    clean_corpus_X_train.append(keras_tokenize(corpus_X_train[i]))
    
clean_corpus_X_train = np.array(clean_corpus_X_train)

  0%|          | 0/127656 [00:00<?, ?it/s]

In [13]:
clean_corpus_X_test = []
for i in notebook.tqdm(range(len(corpus_X_test))):
    clean_corpus_X_test.append(keras_tokenize(corpus_X_test[i]))
    
clean_corpus_X_test = np.array(clean_corpus_X_test)

  0%|          | 0/31915 [00:00<?, ?it/s]

In [14]:
print("Исходный текст: \n", corpus_X_train[0])
print("Очищенный и лемматизированный текст: \n", clean_corpus_X_train[0])

Исходный текст: 
 Grandma Terri Should Burn in Trash 
Grandma Terri is trash. I hate Grandma Terri. F%%K her to HELL! 71.74.76.40
Очищенный и лемматизированный текст: 
 grandma terri should burn in trash grandma terri is trash i hate grandma terri f k her to hell


Создадим переменные для мешков, n-грамм и TF-IDF:

In [15]:
count_vect = CountVectorizer()
count_vect.fit(corpus_X_train)
bow = count_vect.transform(corpus_X_test)
count_vect.fit(clean_corpus_X_train)
bow_cl = count_vect.transform(clean_corpus_X_test)
print("Размер мешка без учёта стоп-слов:", bow.shape)
print("Размер очищенного мешка без учёта стоп-слов:", bow_cl.shape)

Размер мешка без учёта стоп-слов: (31915, 165785)
Размер очищенного мешка без учёта стоп-слов: (31915, 152799)


In [16]:
count_vect = CountVectorizer(stop_words=stop_words)
count_vect.fit(corpus_X_train)
bow_sw = count_vect.transform(corpus_X_test)
count_vect.fit(clean_corpus_X_train)
bow_sw_cl = count_vect.transform(clean_corpus_X_test)
print("Размер мешка с учётом стоп-слов:", bow_sw.shape)
print("Размер очищенного мешка с учётом стоп-слов:", bow_sw_cl.shape)

Размер мешка с учётом стоп-слов: (31915, 165640)
Размер очищенного мешка с учётом стоп-слов: (31915, 152656)


In [17]:
count_vect = CountVectorizer(ngram_range=(2, 2))
count_vect.fit(corpus_X_train)
n_gramm = count_vect.transform(corpus_X_test)
count_vect.fit(clean_corpus_X_train)
n_gramm_cl = count_vect.transform(clean_corpus_X_test)
print("Размер:", n_gramm.shape)
print("Размер очищенных n-грамм:", n_gramm_cl.shape)

Размер: (31915, 1932725)
Размер очищенных n-грамм: (31915, 1870998)


In [18]:
count_tf_idf = TfidfVectorizer(stop_words=stop_words)
count_tf_idf.fit(corpus_X_train)
tf_idf = count_tf_idf.transform(corpus_X_test)
count_tf_idf.fit(clean_corpus_X_train)
tf_idf_cl = count_tf_idf.transform(clean_corpus_X_test)
print("Размер матрицы:", tf_idf.shape)
print("Размер очищенной матрицы:", tf_idf_cl.shape)

Размер матрицы: (31915, 165640)
Размер очищенной матрицы: (31915, 152656)


### BERT

Теперь перейдём к BERT. Для начала создадим модели для токенизации и эмбеддинга (используем *BERT base uncased*):

In [19]:
tokenizer = transformers.BertTokenizerFast.from_pretrained(
    'bert-base-uncased', 
    do_lower_case=True
)
model = transformers.BertModel.from_pretrained(
    'bert-base-uncased',
    num_labels=2,
    output_attentions=False,
    output_hidden_states=False
)
model = model.cuda()

Токенизируем тексты, а затем приведем их к единому размеру и сделаем маски.

In [20]:
tokenized = sample['text'].apply(
    lambda x: tokenizer.encode(x, add_special_tokens=True)
)
max_len = len(max(tokenized.values, key=len))
padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])
attention_mask = np.where(padded != 0, 1, 0)

Token indices sequence length is longer than the specified maximum sequence length for this model (1212 > 512). Running this sequence through the model will result in indexing errors


Обрубим токенизированные тексты по максимальному значению текущей модели BERT:

In [21]:
truncated = padded[:,:512]
truncated_attention_mask = attention_mask[:,:512]

print(truncated[0].shape)
print(truncated_attention_mask[0].shape)

(512,)
(512,)


Теперь преобразуем тексты в эмбеддинги:

In [22]:
batch_size = 128
embeddings = []

for i in notebook.tqdm(range(truncated.shape[0] // batch_size)):
    batch = torch.cuda.LongTensor(truncated[batch_size*i:batch_size*(i+1)])
    attention_mask_batch = torch.cuda.LongTensor(truncated_attention_mask[batch_size*i:batch_size*(i+1)])

    with torch.no_grad():
        batch_embeddings = model(batch, attention_mask=attention_mask_batch)

    embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())

  0%|          | 0/253 [00:00<?, ?it/s]

### Подготовка выборок

Сделаем выборку BERT.

In [23]:
end = batch_size * (truncated.shape[0] // batch_size)
bert_features = pd.DataFrame(np.concatenate(embeddings), index=sample['toxic'][:end].index)
bert_target = sample['toxic'][:end]

del embeddings

И разделим все выборки на трейн и тест.

In [24]:
X = [bow, bow_cl, bow_sw, bow_sw_cl, n_gramm, n_gramm_cl, tf_idf, tf_idf_cl] + [bert_features]
y = [corpus_y_test] * (len(X) - 1) + [bert_target]

X_train, X_test, y_train, y_test = multi_split(X, y, random_state=STATE, stratify=y)

Проверим объявленные переменные.

In [25]:
%who

CatBoostClassifier	 CountVectorizer	 LogisticRegression	 STATE	 TfidfVectorizer	 X	 X_test	 X_train	 attention_mask	 
attention_mask_batch	 batch	 batch_embeddings	 batch_size	 bert_features	 bert_target	 bow	 bow_cl	 bow_sw	 
bow_sw_cl	 clean_corpus_X_test	 clean_corpus_X_train	 clean_text	 corpus_X	 corpus_X_test	 corpus_X_train	 corpus_y	 corpus_y_test	 
corpus_y_train	 count_tf_idf	 count_vect	 device	 df	 end	 f1_score	 i	 keras_tokenize	 
max_len	 model	 multi_split	 n_gramm	 n_gramm_cl	 nltk	 notebook	 np	 ones	 
padded	 pd	 re	 sample	 stop_words	 stopwords	 text_to_word_sequence	 tf_idf	 tf_idf_cl	 
tokenized	 tokenizer	 torch	 train_test_split	 transformers	 truncated	 truncated_attention_mask	 warnings	 y	 
y_test	 y_train	 zeros	 


И очистим память.

In [26]:
model.zero_grad()
torch.cuda.empty_cache()

del X, attention_mask, attention_mask_batch, batch, batch_embeddings, batch_size, bert_features, bert_target, bow, \
bow_cl, bow_sw, bow_sw_cl, clean_corpus_X_test, clean_corpus_X_train, clean_text, corpus_X, corpus_X_test, \
corpus_X_train, corpus_y, corpus_y_test, corpus_y_train, count_tf_idf, count_vect, device, end, i, max_len, model, \
n_gramm, n_gramm_cl, ones, padded, sample, stop_words, tf_idf, tf_idf_cl, tokenized, truncated, \
truncated_attention_mask, y, zeros

## Обучение

Теперь напишем функцию для тестирования модели и на её основе ещё одну, для тестирования на нескольких выборках.

In [27]:
def model_test(clf, X_train, y_train, X_test, y_test, label='', verbose=True):
    clf.fit(X_train, y_train)
    
    train_score = round(f1_score(y_train, clf.predict(X_train)), 4)
    test_score = round(f1_score(y_test, clf.predict(X_test)), 4)
    if verbose:
        print(f'== {clf.__class__.__name__}{label} ==')
        print(f'F1 train: {train_score}')
        print(f'F1 test:  {test_score}')
        print()
    
    return pd.DataFrame(
        [[f'{clf.__class__.__name__}{label}', train_score, test_score]], 
        columns=['model', 'train_score', 'test_score']
    )


def multi_sample_test(clf, X_train, y_train, X_test, y_test, labels=[], params=None, verbose=True):
    stats = pd.DataFrame(columns=['model', 'train_score', 'test_score'])
    
    for i in range(len(X_train)):
        model = clf(**params)
        stats = pd.concat([
            stats, 
            model_test(model, X_train[i], y_train[i], X_test[i], y_test[i], label=labels[i], verbose=verbose)
        ])
        
    return stats.reset_index(drop=True)

Обозначим постфиксы для результатов.

In [28]:
labels = ['_bow', '_bow_cl', '_bow_sw', '_bow_sw_cl', '_n_gramm', '_n_gramm_cl', '_tf_idf', '_tf_idf_cl', '_bert']

Проверим какой результат выдаст логистическая регрессия:

In [29]:
lrm_params = dict(
    max_iter=1000, 
    random_state=STATE
)
stats = multi_sample_test(LogisticRegression, X_train, y_train, X_test, y_test, labels=labels, params=lrm_params)

== LogisticRegression_bow ==
F1 train: 0.9431
F1 test:  0.7298

== LogisticRegression_bow_cl ==
F1 train: 0.9393
F1 test:  0.7257

== LogisticRegression_bow_sw ==
F1 train: 0.9333
F1 test:  0.7206

== LogisticRegression_bow_sw_cl ==
F1 train: 0.9301
F1 test:  0.724

== LogisticRegression_n_gramm ==
F1 train: 0.9352
F1 test:  0.4872

== LogisticRegression_n_gramm_cl ==
F1 train: 0.9299
F1 test:  0.4904

== LogisticRegression_tf_idf ==
F1 train: 0.6459
F1 test:  0.5858

== LogisticRegression_tf_idf_cl ==
F1 train: 0.6522
F1 test:  0.5991

== LogisticRegression_bert ==
F1 train: 0.9083
F1 test:  0.8905



Также проверим BERT выборку на CatBoostClassifier.

In [30]:
cbm = CatBoostClassifier(
    silent=True,
    iterations=100,
    loss_function='Logloss',
    task_type='GPU',
    devices='0'
)
stats = pd.concat([stats, model_test(cbm, X_train[8], y_train[8], X_test[8], y_test[8], label='_bert')])

== CatBoostClassifier_bert ==
F1 train: 0.9298
F1 test:  0.871



## Результаты исследования

### Общие выводы

Укажем пороговое значение метрики f1 и отсортируем результаты в порядке убывания:

In [31]:
stats = pd.concat([stats, pd.DataFrame([['BASELINE', .75, .75]], columns=['model', 'train_score', 'test_score'])])
stats.sort_values(by='test_score', ascending=False).reset_index(drop=True)

,model,train_score,test_score
0,LogisticRegression_bert,0.9083,0.8905
1,CatBoostClassifier_bert,0.9298,0.8710
2,BASELINE,0.7500,0.7500
3,LogisticRegression_bow,0.9431,0.7298
4,LogisticRegression_bow_cl,0.9393,0.7257
5,LogisticRegression_bow_sw_cl,0.9301,0.7240
6,LogisticRegression_bow_sw,0.9333,0.7206
7,LogisticRegression_tf_idf_cl,0.6522,0.5991
8,LogisticRegression_tf_idf,0.6459,0.5858
9,LogisticRegression_n_gramm_cl,0.9299,0.4904


В этом исследовании мы обучали модель для предсказания токсичности текста.

Для работы нам предоставили датасет с текстами и целевым признаком (токсичный текст или нет).

Сначала мы подготовили корпус и сделали из него несколько выборок: мешки, биграммы, TF-IDF.

Далее мы провели работу с моделью BERT, токенизировали тексты и преобразовали их в эмбеддинги.

После этого мы создали выборки и обучили на них логистическую регрессию. Кроме того, на BERT-выборке мы обучили и Catboost.

Требуемый результат выше .75 f1 показали только модели, обученные на эмбеддингах BERT. При этом логистическая регрессия показала налучший результат. Остальные модели не смогли достигнуть требуемого результата.

В итоге, для предсказания токсичности текста можно использовать логистическую регрессию, обученную на эмбеддингах BERT. Однако, вероятно, при более тщательном подборе параметров и другие варианты смогутдостичь требуемого результата.

### Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны